En este script obtenemos las respuestas de los clickbaits mediante el modelo por similitud, extrayendo la frase del contenido que mayor similitud tenga con el titular de la noticia.

In [ ]:
!pip install sentence-transformers
import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import json
import nltk
from sentence_transformers import SentenceTransformer, util

nltk.download("punkt_tab")
from nltk.tokenize import sent_tokenize

# Carga el modelo de embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_best_sentence(title, content):
    sentences = sent_tokenize(content)
    embeddings = model.encode([title] + sentences, convert_to_tensor=True)
    similarities = util.cos_sim(embeddings[0], embeddings[1:])[0]
    best_idx = similarities.argmax().item()
    return sentences[best_idx]

def prepare_squad_data_auto(df, title_col='title', content_col='content'):
    squad_format = {"data": []}
    for i, row in df.iterrows():
        title = str(row[title_col])
        content = str(row[content_col])

        if not title or not content:
            continue

        best_sentence = extract_best_sentence(title, content)

        if best_sentence not in content:
            continue  # Sanity check

        answer_start = content.find(best_sentence)

        qa = {
            "context": content,
            "question": title,
            "id": str(i),
            "answers": {
                "text": [best_sentence],
                "answer_start": [answer_start]
            }
        }

        squad_format["data"].append({
            "title": "dataset_clickbait_QA_limpio",
            "paragraphs": [qa]
        })

    return squad_format

# Carga el CSV y aplica
df = pd.read_csv("dataset_clickbait_QA_limpio.csv")
squad_data = prepare_squad_data_auto(df)

# Guarda como JSON
with open("clickbait_autoqa.json", "w", encoding='utf-8') as f:
    json.dump(squad_data, f, indent=2, ensure_ascii=False)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import json
import pandas as pd

# Carga el JSON generado previamente
with open("clickbait_autoqa.json", "r", encoding="utf-8") as f:
    squad_data = json.load(f)

# Extrae datos y construye el DataFrame
rows = []
for item in squad_data["data"]:
    for para in item["paragraphs"]:
        context = para["context"]
        question = para["question"]
        answer_text = para["answers"]["text"][0]
        answer_start = para["answers"]["answer_start"][0]
        rows.append({
            "title": question,
            "content": context,
            "answer_text": answer_text,
            "answer_start": answer_start
        })

df = pd.DataFrame(rows)
df.to_csv("qa_output_similitud.csv", index=False, encoding="utf-8")